# Clean up the big .csv file for easier use with my other codes

### Run the compileOCdata.ipynb first

I will use RA, Dec, coordinates from Simbad when possible.

The final file should have columns with IDs from all sources (as available), RA, Dec, distance, Age, Metallicity, Mass, Rhm (all as available)

In [1]:
import pandas as pd
import numpy as np
import sys
import time
import re

from astropy.coordinates import SkyCoord
from astropy import units 

from astroquery.simbad import Simbad
Simbad.TIMEOUT = 300 # sets the timeout to 60s

import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 50)

%matplotlib inline

In [ ]:
matchCoords = True

## Get the coordinates

In [13]:
def getSimbadRADec(match_ID):
    #I'd like to use the Simbad names for matching (if possible) so I reduce the duplicates, 

    RA = np.full(len(match_ID), np.nan, dtype='object')
    Dec = np.full(len(match_ID), np.nan, dtype='object')


#     result_table = Simbad.query_objects(match_ID) #stupidly, this does not return blank rows for missing data!
# So I'm trying to go one line at a time, but of course Simbad doesn't like that... it rejects my connection when
# I have too many requests (and what is too many?  and how long should I wait?)
    for i, ID in enumerate(match_ID):
        result_table = Simbad.query_object(ID)
        if (result_table):
            #print(result_table.columns)
            if (len(result_table) > 0):
                row = result_table[0]
                RA[i] = row['RA']
                Dec[i] = row['DEC']
        print(f'{i} {float(i)/len(match_ID):5.3f}, {ID} {RA[i]} {Dec[i]} -- ', end='')
        if (i>0 and i % 50 == 0):
            time.sleep(30)
            print('\n\n')
        time.sleep(0.1)

        
    return RA, Dec

In [5]:
def getCoord(row):

    if (pd.notna(row['_RAJ2000_cantat-gaudin']) and pd.notna(row['_DEJ2000_cantat-gaudin'])):
        return SkyCoord(ra=row['_RAJ2000_cantat-gaudin']+' hours', dec=row['_DEJ2000_cantat-gaudin']+' degree', frame='icrs')
    
    elif (pd.notna(row['ra_mwsc']) and pd.notna(row['dec_mwsc'])):
        return SkyCoord(ra=row['ra_mwsc']+' hours', dec=row['dec_mwsc']+' degree', frame='icrs')
        
    elif (pd.notna(row['RA[hr]_kharchenko']) and pd.notna(row['Dec[deg]_kharchenko'])):
        return SkyCoord(ra=row['RA[hr]_kharchenko']*units.hourangle, dec=row['Dec[deg]_kharchenko']*units.degree, frame='icrs')

    elif (pd.notna(row['GLON[deg]_piskunov']) and pd.notna(row['GLAT[deg]_piskunov'])):
        return SkyCoord(l=row['GLON[deg]_piskunov']*units.degree, b=row['GLAT[deg]_piskunov']*units.degree, frame='galactic').icrs

    elif (pd.notna(row['ra_lynga']) and pd.notna(row['dec_lynga'])):
        return SkyCoord(ra=row['ra_lynga']*units.degree, dec=row['dec_lynga']*units.degree, frame='icrs')
    
    elif (pd.notna(row['_RA[deg]_vandenbergh']) and pd.notna(row['_Dec[deg]_vandenbergh'])):
        return SkyCoord(ra=row['_Dec[deg]_vandenbergh']*units.degree, dec=row['_Dec[deg]_vandenbergh']*units.degree, frame='icrs')

    elif (pd.notna(row['RA_2000_webda']) and pd.notna(row['Dec_2000_webda'])):
        return SkyCoord(ra=row['RA_2000_webda']+' hours', dec=row['Dec_2000_webda']+' degree', frame='icrs')
            
    elif (pd.notna(row['GLON[deg]_cantat-gaudin']) and pd.notna(row['GLAT[deg]_cantat-gaudin'])):
        return SkyCoord(l=row['GLON[deg]_cantat-gaudin']*units.degree, b=row['GLAT[deg]_cantat-gaudin']*units.degree, frame='galactic').icrs

    elif (pd.notna(row['l_webda']) and pd.notna(row['b_webda'])):
        return SkyCoord(l=row['l_webda']*units.degree, b=row['b_webda']*units.degree, frame='galactic').icrs
    
    else:
        print('NO RA, Dec : ', row['Simbad_ID'])
        return False

In [3]:
OC_df = pd.read_csv('OCcompiled.csv')
OC_df

,Simbad_ID,Name_mwsc,broad_type_mwsc,cluster_status_mwsc,ra_mwsc,dec_mwsc,lii_mwsc,bii_mwsc,core_radius_mwsc,central_radius_mwsc,cluster_radius_mwsc,pm_ra_mwsc,pm_dec_mwsc,pm_tot_error_mwsc,rad_vel_mwsc,rad_vel_error_mwsc,num_rad_vel_stars_mwsc,num_core_stars_mwsc,num_central_stars_mwsc,num_cluster_stars_mwsc,distance_mwsc,e_bv_mwsc,distance_modulus_mwsc,e_jk_mwsc,e_jh_mwsc,delta_h_mwsc,log_age_mwsc,log_age_error_mwsc,num_log_age_stars_mwsc,king_core_radius_mwsc,king_core_radius_error_mwsc,king_tidal_radius_mwsc,king_tidal_radius_error_mwsc,king_norm_factor_mwsc,king_norm_factor_error_mwsc,reference_code_mwsc,cluster_type_mwsc,metallicity_mwsc,metallicity_error_mwsc,num_metallicity_stars_mwsc,comments_mwsc,class_mwsc,Name_webda,RA_2000_webda,Dec_2000_webda,l_webda,b_webda,Dist_webda,Mod_webda,EB-V_webda,Age_webda,ST_webda,Z_webda,Diam_webda,Fe/H_webda,MRV_webda,pm RA_webda,pm Dec_webda,Measures_webda,Stars_webda,COCD_piskunov,Name_piskunov,GLON[deg]_piskunov,GLAT[deg]_piskunov,DistMod_piskunov,E(B-V)_piskunov,Dist[pc]_piskunov,logt[yr]_piskunov,rt[pc]_piskunov,e_rt[pc]_piskunov,logM[MSun]_piskunov,e_logM[MSun]_piskunov,rtA[pc]_piskunov,e_rtA[pc]_piskunov,logMA[MSun]_piskunov,e_logMA[MSun]_piskunov,MWSC_kharchenko,Name_kharchenko,Type_kharchenko,n_Type_kharchenko,RA[hr]_kharchenko,Dec[deg]_kharchenko,GLON[deg]_kharchenko,GLAT[deg]_kharchenko,r0[deg]_kharchenko,r1[deg]_kharchenko,r2[deg]_kharchenko,pmRA[mas/yr]_kharchenko,pmDec[mas/yr]_kharchenko,e_pm[mas/yr]_kharchenko,RV[km/s]_kharchenko,e_RV[km/s]_kharchenko,n_RV[km/s]_kharchenko,N1sr0_kharchenko,N1sr1_kharchenko,N1sr2_kharchenko,d[pc]_kharchenko,E(B-V)_kharchenko,appDistMod[mag]_kharchenko,E(J-Ks)_kharchenko,E(J-H)_kharchenko,dH_kharchenko,logt[yr]_kharchenko,e_logt[yr]_kharchenko,Nt_kharchenko,rc[pc]_kharchenko,e_rc[pc]_kharchenko,rt[pc]_kharchenko,e_rt[pc]_kharchenko,k[pc-2]_kharchenko,e_k[pc-2]_kharchenko,Src_kharchenko,SType_kharchenko,[Fe/H][Sun]_kharchenko,e_[Fe/H][Sun]_kharchenko,n_[Fe/H]_kharchenko,Name_salaris,dV_salaris,err_dV_salaris,[Fe/H]_salaris,err_[Fe/H]_salaris,t[Gyr]_salaris,err_t_salaris,Rgc[kpc]_salaris,z[pc]_salaris,flag_salaris,tJP94[Gyr]_salaris,Seq_vandenbergh,Name_vandenbergh,l[deg]_vandenbergh,Diam[pc]_vandenbergh,R[pc]_vandenbergh,Z[pc]_vandenbergh,E(B-V)_vandenbergh,logT[yr]_vandenbergh,SimbadName_vandenbergh,_RA[deg]_vandenbergh,_Dec[deg]_vandenbergh,_RAJ2000_cantat-gaudin,_DEJ2000_cantat-gaudin,Name_cantat-gaudin,RAJ2000[deg]_cantat-gaudin,DEJ2000[deg]_cantat-gaudin,GLON[deg]_cantat-gaudin,GLAT[deg]_cantat-gaudin,r50[deg]_cantat-gaudin,Nstars_cantat-gaudin,pmRA[mas/yr]_cantat-gaudin,pmDE[mas/yr]_cantat-gaudin,plx[mas]_cantat-gaudin,dmode[pc]_cantat-gaudin,Rgc[pc]_cantat-gaudin,SimbadName_cantat-gaudin,Name_lynga,ra_lynga,dec_lynga,distance_lynga,log_age_lynga,angular_diameter_lynga,alt_name_lynga,lii_lynga,bii_lynga,iau_num_lynga,seq_code_lynga,seq_num_lynga,prec_ra_lynga,prec_dec_lynga,lund_record_num_lynga,ocl_num_lynga,ref_angular_diameter_lynga,ref_distance_lynga,ref_log_age_lynga,metallicity_lynga,ref_metallicity_lynga,e_bv_lynga,ref_e_bv_lynga,type_flag_lynga,ref_type_flag_lynga,tr_concent_class_lynga,tr_range_class_lynga,tr_richness_class_lynga,tr_nebulosity_lynga,sb_bs_mag_lynga,sb_spect_code_lynga,sb_total_mag_lynga,sk_total_mag_lynga,sk_bv_color_lynga,sk_num_stars_lynga,ja_star_num_lynga,ja_class_lynga,ja_max_class_lynga,ja_richness_lynga,ja_e_bv_lynga,ref_ja_e_bv_lynga,ja_bv_turnoff_lynga,ref_ja_bv_turnoff_lynga,ly_tr_concent_class_lynga,ly_tr_range_class_lynga,ly_tr_richness_class_lynga,ly_tr_nebulosity_lynga,ly_member_stars_lynga,ly_angular_diameter_lynga,ly_refs_flag_lynga,radvel_weight_lynga,radvel_lynga,radvel_weight_class_lynga,ref_radvel1_lynga,ref_radvel2_lynga,ref_radvel3_lynga,ref_radvel4_lynga,ref_radvel5_lynga,basel_spect_code_lynga,basel_color_type_lynga,neg_ra_tracer_lynga,neg_lii_tracer_lynga,neg_seq_num_tracer_lynga,pos_ra_tracer_lynga,pos_lii_tracer_lynga,pos_seq_num_tracer_lynga,jdl_distance_lynga,jdl_distance_weight

In [ ]:
#this takes a while...
if (matchCoords):
    RA_Simbad, Dec_Simbad = getSimbadRADec(OC_df['Simbad_ID'].values)

0 0.000, [KPS2012]_MWSC_4688 23 51 54 -86 43.2 -- 1 0.000, [KPS2012]_MWSC_5684 nan nan -- 2 0.001, [KPS2012]_MWSC_5692 nan nan -- 3 0.001, [KPS2012]_MWSC_4005 00 11 28 -85 28.8 -- 4 0.001, [KPS2012]_MWSC_4176 14 27 18 -85 25.2 -- 5 0.001, ESO_8-6 14 56 54 -83 26.7 -- 6 0.002, [KPS2012]_MWSC_4219 15 42 14 -83 11.7 -- 7 0.002, [KPS2012]_MWSC_5575 nan nan -- 8 0.002, [KPS2012]_MWSC_4682 23 43 23 -82 57.6 -- 9 0.003, [KPS2012]_MWSC_5685 nan nan -- 10 0.003, [KPS2012]_MWSC_5681 nan nan -- 11 0.003, [KPS2012]_MWSC_2014 12 20 06 -81 30.6 -- 12 0.004, [KPS2012]_MWSC_2064 12 45 43 -81 15.5 -- 13 0.004, [KPS2012]_MWSC_2049 12 34 55 -80 57.0 -- 14 0.004, [KPS2012]_MWSC_5688 nan nan -- 15 0.004, ESO_26-2 20 26 38 -80 00.0 -- 16 0.005, [KPS2012]_MWSC_4137 09 26 12 -79 44.4 -- 17 0.005, Cl_Melotte_227 20 16 48 -79 02.4 -- 18 0.005, [KPS2012]_MWSC_5679 nan nan -- 19 0.006, [KPS2012]_MWSC_5572 nan nan -- 20 0.006, [KPS2012]_MWSC_5749 nan nan -- 21 0.006, ESO_21-6 14 15 52 -78 30.9 -- 22 0.007, [KPS201

151 0.045, Cl_Hogg_15 12 43 36 -63 05.9 -- 152 0.045, [KPS2012]_MWSC_2134 13 33 50 -63 02.4 -- 153 0.046, C_1206-626 12 09 45.0 -62 59 17 -- 154 0.046, Cl_Dias_4 13 43 25 -63 00.0 -- 155 0.046, NGC_4609 12 42 18 -62 59.7 -- 156 0.046, IC_2944 11 38 20.00 -63 22 22.0 -- 157 0.047, NGC_5281 13 46 35 -62 55.0 -- 158 0.047, Ass_Cir_OB_1 14 45 -62.9 -- 159 0.047, [DBS2003]_129 12 15 24 -62 55.1 -- 160 0.048, NGC_5269 13 44 44.0 -62 54 54 -- 161 0.048, Loden_1373 nan nan -- 162 0.048, [KPS2012]_MWSC_5696 nan nan -- 163 0.049, Cl_Trumpler_21 13 32.2 -62 48 -- 164 0.049, IC_2714 11 17 27 -62 44.0 -- 165 0.049, NGC_4184 12 13 34 -62 43.8 -- 166 0.049, [KPS2012]_MWSC_1959 11 36 58 -62 43.4 -- 167 0.050, C_1154-623 11 57 28 -62 43.0 -- 168 0.050, Cl_Danks_1 13 12 27 -62 42.1 -- 169 0.050, NGC_3036 09 49.2 -62 40 -- 170 0.051, Cl_Danks_2 13 12 55 -62 40.9 -- 171 0.051, [DBS2003]_85 13 18 31 -62 39.8 -- 172 0.051, [FSR2007]_1595 11 47 05.0 -62 38 13 -- 173 0.052, [KPR2005]_89 17 23.3 -62 38 -- 174 

301 0.090, [KPR2005]_62 10 50 53 -60 06.0 -- 302 0.090, C_1045-598 10 47 15.2 -60 05 51 -- 303 0.090, NGC_5043 13 16.1 -60 04 -- 304 0.091, [KPS2012]_MWSC_2219 14 28 34 -60 01.2 -- 305 0.091, [KPS2012]_MWSC_5723 nan nan -- 306 0.091, Cl_Ruprecht_111 14 36 00 -59 58.8 -- 307 0.091, [KPS2012]_MWSC_4116 06 19 12 -59 58.2 -- 308 0.092, NGC_5299 nan nan -- 309 0.092, NGC_2132 05 55 19 -59 55.2 -- 310 0.092, Cl_Collinder_223 10 32 16 -60 01.2 -- 311 0.093, [DBS2003]_89 14 45 03 -59 49.5 -- 312 0.093, Cl_Feinstein_1 11 06 43 -59 49.2 -- 313 0.093, C_1225-595 12 28 44 -59 48.6 -- 314 0.094, Loden_821 nan nan -- 315 0.094, Loden_1095 nan nan -- 316 0.094, Loden_1194 nan nan -- 317 0.094, Loden_1225 nan nan -- 318 0.095, ESO_130-13 12 23 04 -59 39.9 -- 319 0.095, NAME_HD_135159_Group 15 16 44 -59 39.3 -- 320 0.095, NGC_5606 14 27 47 -59 38.4 -- 321 0.096, [KPS2012]_MWSC_2289 15 20 58 -59 37.2 -- 322 0.096, [DBS2003]_51 10 32 58 -59 37.4 -- 323 0.096, NGC_5381 14 00 40 -59 36.3 -- 324 0.097, NGC_

451 0.134, Loden_189 nan nan -- 452 0.135, [DBS2003]_141 15 28 32 -56 22.5 -- 453 0.135, [KPS2012]_MWSC_2109 13 19 47 -56 21.1 -- 454 0.135, C_0920-560 09 21 55 -56 19.0 -- 455 0.136, C_0949-560 09 51 36 -56 18.6 -- 456 0.136, Cl_VDBH_58 09 10 12.0 -56 15 43 -- 457 0.136, [KPS2012]_MWSC_1689 09 33 56 -56 14.9 -- 458 0.136, [KPS2012]_MWSC_2107 13 19 37 -56 10.8 -- 459 0.137, [KPS2012]_MWSC_1789 10 18 59 -56 10.2 -- 460 0.137, [KPS2012]_MWSC_2025 12 24 16 -56 09.9 -- 461 0.137, [KPS2012]_MWSC_2233 14 34 37 -56 06.9 -- 462 0.138, ESO_165-9 09 05.2 -55 58 -- 463 0.138, NAME_SCHUSTER_CL 10 04 36 -55 51.4 -- 464 0.138, Loden_1 nan nan -- 465 0.139, [KPS2012]_MWSC_1716 09 48 36 -55 47.4 -- 466 0.139, NGC_3960 11 50 33 -55 40.4 -- 467 0.139, NGC_5823 15 05 31 -55 35.7 -- 468 0.139, [KPR2005]_66 11 13.6 -55 25 -- 469 0.140, C_1002-552 10 04 18.00 -55 26 00.0 -- 470 0.140, [SSW94]_c 10 41.5 -55 18 -- 471 0.140, Cl_Lynga_4 15 33 20 -55 14.4 -- 472 0.141, [KPS2012]_MWSC_1818 10 31 36 -55 09.0 -- 4

601 0.179, ESO_211-9 09 16 44 -50 17.0 -- 602 0.179, [KPS2012]_MWSC_1730 09 56 24 -50 16.2 -- 603 0.180, ESO_211-3 08 51 36 -50 14.7 -- 604 0.180, Cl_Lynga_8 16 20 04 -50 13.0 -- 605 0.180, C_0930-499 09 31 52 -50 12.9 -- 606 0.181, [KPS2012]_MWSC_2321 15 48 40 -50 12.9 -- 607 0.181, [DBS2003]_164 16 19 23 -50 09.5 -- 608 0.181, [KPS2012]_MWSC_2420 16 30 25 -50 07.5 -- 609 0.181, [KPS2012]_MWSC_1511 08 37 01 -50 02.4 -- 610 0.182, NAME_HD_80077_Group 09 15 53.0 -50 01 00 -- 611 0.182, [DBS2003]_166 16 21 27 -50 00.7 -- 612 0.182, IC_4651 17 24 49 -49 56.0 -- 613 0.183, [KPS2012]_MWSC_4545 20 07 50 -49 51.6 -- 614 0.183, [DBS2003]_102 16 15 01 -49 50.7 -- 615 0.183, NGC_6167 16 34 34 -49 46.3 -- 616 0.184, [KPS2012]_MWSC_2195 14 00 18 -49 45.9 -- 617 0.184, [KPS2012]_MWSC_2419 16 29 49 -49 45.1 -- 618 0.184, [KPS2012]_MWSC_1640 09 14 40 -49 44.7 -- 619 0.184, [DBS2003]_170 16 28 58.0 -49 36 27 -- 620 0.185, Cl_VDBH_85 10 01 52 -49 35.1 -- 621 0.185, [DBS2003]_167 16 23 24 -49 32.5 -- 62

...

1201 0.358, Dutra-Cl_Bica_60 nan nan -- 1202 0.358, ESO_525-8 19 27 11 -23 35.4 -- 1203 0.358, Cl_Ruprecht_139 18 01.0 -23 32 -- 1204 0.359, [KPS2012]_MWSC_1601 08 56 13 -23 26.1 -- 1205 0.359, C_0728-232 07 30 36 -23 23.0 -- 1206 0.359, C_0734-232 07 36 47 -23 21.3 -- 1207 0.360, NGC_6546 18 07 22 -23 17.8 -- 1208 0.360, [KPS2012]_MWSC_0860 06 25 05 -23 18.6 -- 1209 0.360, [KPS2012]_MWSC_1440 08 10 46 -23 18.3 -- 1210 0.361, [KPS2012]_MWSC_5311 nan nan -- 1211 0.361, [KPS2012]_MWSC_2751 17 56 26 -23 11.5 -- 1212 0.361, Cl_Ruprecht_17 07 23 38 -23 11.1 -- 1213 0.361, [KPR2005]_39 07 33.0 -22 57 -- 1214 0.362, [KPS2012]_MWSC_5717 nan nan -- 1215 0.362, [FSR2007]_0023 17 57 35.0 -22 52 32 -- 1216 0.362, ESO_561-5 07 59.3 -22 41 -- 1217 0.363, C_0716-225 07 17 58 -22 40.0 -- 1218 0.363, M_21 18 04 13 -22 29.4 -- 1219 0.363, Cl_Ruprecht_39 07 52 19 -22 26.8 -- 1220 0.364, [DBS2003]_10 07 35 27 -22 23.9 -- 1221 0.364, [KPS2012]_MWSC_5316 nan nan -- 1222 0.364, [DBS2003]_13 08 12 15 -22 20.9

1351 0.403, C_1829-160 18 32 06 -16 00.6 -- 1352 0.403, Ass_Sgr_OB_6 18 12 -16.0 -- 1353 0.403, [KPS2012]_MWSC_2489 16 55 44 -15 59.7 -- 1354 0.403, [KPS2012]_MWSC_1109 07 07 22 -15 58.8 -- 1355 0.404, [KPS2012]_MWSC_5312 nan nan -- 1356 0.404, [KPS2012]_MWSC_1269 07 34 31 -15 56.1 -- 1357 0.404, [KPS2012]_MWSC_1163 07 17 11 -15 47.7 -- 1358 0.405, [KPS2012]_MWSC_2942 18 32 05 -15 41.2 -- 1359 0.405, Cl_Ruprecht_26 07 37 12 -15 39.0 -- 1360 0.405, NGC_2360 07 17 43 -15 38.5 -- 1361 0.406, [KPS2012]_MWSC_2609 17 29 46 -15 38.1 -- 1362 0.406, NGC_2414 07 33 12 -15 27.2 -- 1363 0.406, [KPS2012]_MWSC_1252 07 31 11 -15 25.7 -- 1364 0.406, C_0726-153 07 28 22 -15 24.3 -- 1365 0.407, [KPS2012]_MWSC_1122 07 08 28 -15 21.9 -- 1366 0.407, C_0726-152 07 28 35 -15 21.9 -- 1367 0.407, [DBS2003]_1 07 30 40 -15 18.0 -- 1368 0.408, [KPS2012]_MWSC_4572 20 31 00 -15 10.5 -- 1369 0.408, [FSR2007]_0070 19 30 02.0 -15 10 02 -- 1370 0.408, [KPS2012]_MWSC_5319 nan nan -- 1371 0.409, Cl_Waterloo_7 07 26 08.0 

1501 0.447, [KPS2012]_MWSC_1390 07 57 47 -09 09.0 -- 1502 0.448, [KPS2012]_MWSC_2659 17 39 52 -09 06.6 -- 1503 0.448, [KPS2012]_MWSC_1048 06 59 20 -09 05.0 -- 1504 0.448, NGC_6728 18 58.8 -08 58 -- 1505 0.448, [KPS2012]_MWSC_0748 06 08 53 -08 56.8 -- 1506 0.449, [KPS2012]_MWSC_0827 06 16 31 -08 56.7 -- 1507 0.449, C_1633-088 16 36 25 -08 56.1 -- 1508 0.449, [KPS2012]_MWSC_5321 nan nan -- 1509 0.450, [KPS2012]_MWSC_1111 07 07 30 -08 51.9 -- 1510 0.450, [IBP2002]_CC04 07 00 32 -08 52.0 -- 1511 0.450, [FSR2007]_0041 17 03 30.0 -08 51 13 -- 1512 0.451, [KPS2012]_MWSC_1036 06 57 55 -08 44.7 -- 1513 0.451, [KPS2012]_MWSC_2846 18 13 48 -08 42.5 -- 1514 0.451, NGC_2349 07 10.8 -08 36 -- 1515 0.451, [KPS2012]_MWSC_2855 18 15 07 -08 27.0 -- 1516 0.452, Cl_Trumpler_34 18 39 48 -08 24.9 -- 1517 0.452, Cl_Dias_3 07 10 23 -08 23.7 -- 1518 0.452, M_50 07 02 47.5 -08 20 16 -- 1519 0.453, NGC_6664 18 36 37 -07 48.8 -- 1520 0.453, NAME_IRAS_06548-0815_Cluster 06 57 15 -08 19.8 -- 1521 0.453, [KPS2012]_M

1651 0.492, C_0646+004 06 48 54 +00 23.0 -- 1652 0.492, [KPS2012]_MWSC_5748 nan nan -- 1653 0.493, NGC_2112 05 53 45 +00 24.6 -- 1654 0.493, [KPS2012]_MWSC_5022 nan nan -- 1655 0.493, NGC_2301 06 51 45 +00 27.6 -- 1656 0.493, [KPS2012]_MWSC_0571 05 32 26 +00 31.8 -- 1657 0.494, [FSR2007]_0098 18 47 31.0 +00 36 51 -- 1658 0.494, [KPR2005]_22 06 14.5 +00 38 -- 1659 0.494, [KPS2012]_MWSC_1038 06 58 16 +00 53.0 -- 1660 0.495, [KPR2005]_18 05 26 10 +00 49.2 -- 1661 0.495, NGC_2324 07 04 07 +01 02.7 -- 1662 0.495, NGC_2189 06 12 07 +01 03.3 -- 1663 0.496, NGC_2262 06 39 40 +01 08.4 -- 1664 0.496, [KPS2012]_MWSC_1407 08 00 20 +01 15.6 -- 1665 0.496, [KPS2012]_MWSC_4317 17 10 28 +01 15.9 -- 1666 0.496, C_1745+013 17 48 06 +01 18.0 -- 1667 0.497, Teutsch_13 06 43 55.0 +01 24 09 -- 1668 0.497, [KPS2012]_MWSC_4547 20 12 13 +01 31.5 -- 1669 0.497, [KPS2012]_MWSC_0947 06 39 58 +01 32.0 -- 1670 0.498, [KPS2012]_MWSC_3054 19 09 03 +01 32.2 -- 1671 0.498, [KPS2012]_MWSC_1140 07 11 37 +01 36.3 -- 1672 

1801 0.537, NGC_2312 06 58 48 +10 16.8 -- 1802 0.537, NGC_6709 18 51 18 +10 19.1 -- 1803 0.537, NGC_6724 18 56 45.00 +10 26 47.0 -- 1804 0.538, [KPS2012]_MWSC_5645 nan nan -- 1805 0.538, NGC_2141 06 02 55 +10 26.8 -- 1806 0.538, [KPS2012]_MWSC_4602 21 02 46 +10 45.9 -- 1807 0.538, NAME_Poole_J1856+10.8 18 55 53 +10 46.4 -- 1808 0.539, NAME_Poole_J1855+10.8 18 55 08 +10 46.5 -- 1809 0.539, NGC_2259 06 38 34.4 +10 53 24 -- 1810 0.539, NGC_1662 04 48 27 +10 56.2 -- 1811 0.540, NGC_6525 18 02 00.0 +11 01 24 -- 1812 0.540, [FSR2007]_0979 06 31 18.0 +11 04 51 -- 1813 0.540, [KPS2012]_MWSC_2724 17 51 16 +11 09.7 -- 1814 0.541, NGC_6858 20 02 56.0 +11 15 30 -- 1815 0.541, Cl_Berkeley_43 19 15 30 +11 15.9 -- 1816 0.541, [KPS2012]_MWSC_0626 05 40 07 +11 27.6 -- 1817 0.541, Dol-Dzim_2 nan nan -- 1818 0.542, [KPS2012]_MWSC_0535 05 25 32 +11 35.6 -- 1819 0.542, NGC_6738 19 01.3 +11 37 -- 1820 0.542, Cl_Dias_8 19 52 10 +11 37.7 -- 1821 0.543, C_1924+115 19 25 25 +11 38.1 -- 1822 0.543, NGC_6837 19 5

In [ ]:
#for those without Simbad matches, use the position from the compiled table
if (matchCoords):
    RA = RA_Simbad
    Dec = Dec_Simbad
    for index, row in OCs.iterrows():
        if (pd.isna(RA_Simbad[index])):
            #RA and Dec
            c = getCoord(row)
            if (c):
                RA[index] = c.ra.hms #need to format this properly
                Dec[index] = c.dec.dms



In [ ]:
#write this to an intermediate file so that I don't need to match to Simbad again!
if (matchCoords):
    cleanOC_df = pd.DataFrame()
    cleanOC_df['ID'] = OC_df['Simbad_ID'] #note: this contains some IDs that are not in Simbad
    ############# Remove this later
    RA = RA_Simbad
    Dec = Dec_Simbad
    #############
    cleanOC_df['RA'] = RA
    cleanOC_df['Dec'] = Dec
    cleanOC_df['Simbad_RA'] = RA_Simbad
    cleanOC_df['Simbad_Dec'] = Dec_Simbad

    cleanOC_df.to_csv('OCcompiled_clean.csv', index=False)

*Below, we can work with this cleanOC_df so that I don't have to go through the Simbad matching again!*

In [ ]:
cleanOC_df = pd.read_csv('OCcompiled_clean.csv')
cleanOC_df

### Add all the IDs

In [ ]:
#add a column with the actual Simbad ID in the cleaned table, with NaN when there isn't a match
simbad_ID = cleanOC_df['ID'].values
for index, row in  cleanOC_df.iterrows():
    if (pd.isna(row['Simbad_RA'])):
        simbad_ID[i] = np.nan

cleanOC_df['ID_Simbad'] = simbad_ID
cleanOC_df['ID_MWSC'] = OC_df['Name_mwsc']
cleanOC_df['ID_WEBDA'] = OC_df['Name_webda']
cleanOC_df['ID_Piskunov'] = OC_df['Name_piskunov']
cleanOC_df['ID_Kharchenko'] = OC_df['Name_kharchenko']
cleanOC_df['ID_Salaris'] = OC_df['Name_salaris']
cleanOC_df['ID_vandenBergh'] = OC_df['Name_vemndenbergh']
cleanOC_df['ID_Cantat-Gaudin'] = OC_df['Name_cantat-gaudin']
cleanOC_df['ID_Lynga'] = OC_df['Name_lynga']

In [ ]:
### Masses

In [ ]:
def getMass(row, mm = 0.5):
    #take a mean if there are more than 1
    masses = []
    if (pd.notna(row['num_cluster_stars'])):
        masses.append(row['num_cluster_stars']*mm)
        
    if (pd.notna(row['Nstars'])):
        masses.append(row['Nstars']*mm)    

    if (pd.notna(row['Stars'])):
        masses.append(row['Stars']*mm)    
        
    if (pd.notna(row['ly_member_stars'])):
        if row['ly_member_stars'].isnumeric():
            masses.append(float(row['ly_member_stars'])*mm)
        
    if (pd.notna(row['N1sr2'])):
        masses.append(row['N1sr2']*mm)    
        
    if (pd.notna(row['logM[MSun]'])):
        masses.append(10.**row['logM[MSun]'])
        
    if (pd.notna(row['logMA[MSun]'])):
        masses.append(10.**row['logMA[MSun]'])
        

    if (len(masses) > 0):
        masses = np.array(masses)
        return (np.mean(masses), np.std(masses)/(len(masses))**0.5)
    else:
        print('NO MASS', row['Name'])
        return (np.nan, np.nan)

mass = []
for index, row in OCs.iterrows():
    m,em  = getMass(row)
    if (~np.isnan(m)):
        mass.append(m)
        

# hasAge = OCs.loc[(pd.notna(OCs['Age'])) | 
#                  (pd.notna(OCs['log_age'])) |
#                  (pd.notna(OCs['logt[yr]'])) |
#                  (pd.notna(OCs['logt[yr]_kharchenko'])) |
#                  (pd.notna(OCs['logt'])) |
#                  (pd.notna(OCs['logt_vandenbergh'])) |
#                  (pd.notna(OCs['t[Gyr]']))
#                  ]
# hasBoth = OCs.loc[( 
#          (pd.notna(OCs['logM[MSun]'])) |
#          (pd.notna(OCs['num_cluster_stars'])) |
#          (pd.notna(OCs['Stars'])) | 
#          (pd.notna(OCs['logMA[MSun]'])) |
#          (pd.notna(OCs['N1sr0'])) |
#          (pd.notna(OCs['N1sr1'])) |
#          (pd.notna(OCs['N1sr2'])) 
#     ) & (
#         (pd.notna(OCs['Age'])) | 
#         (pd.notna(OCs['log_age'])) |
#         (pd.notna(OCs['logt[yr]'])) |
#         (pd.notna(OCs['logt[yr]_kharchenko'])) |
#         (pd.notna(OCs['logt'])) |
#         (pd.notna(OCs['logt_vandenbergh'])) |
#         (pd.notna(OCs['t[Gyr]']))
#     )]
print(len(OCs), len(mass))
#print(mass)

In [ ]:
noAge = OCs.loc[(pd.isnull(OCs['Age'])) &
                 (pd.isnull(OCs['log_age'])) &
                 (pd.isnull(OCs['logt[yr]'])) &
                 (pd.isnull(OCs['logt[yr]_kharchenko'])) &
                 (pd.isnull(OCs['logt'])) &
                 (pd.isnull(OCs['logt_vandenbergh'])) &
                 (pd.isnull(OCs['t[Gyr]']))
                 ]
noAge

In [ ]:
#Add a column to estimate the mass from the number of stars? (or vice versa) 
#This would require an estimate of the mean mass, which depends on age

#as a test, just assume <m>=0.5
meanM = 0.5

logMass = []
logAge = []
name = []
for index, row in  hasBoth.iterrows():
    name.append(row['name'])
    
    #age
#     (pd.notnull(OCs['Age'])) | 
#                  (pd.notnull(OCs['log_age'])) |
#                  (pd.notnull(OCs['logt[yr]'])) |
#                  (pd.notnull(OCs['logt[yr]_kharchenko'])) |
#                  (pd.notnull(OCs['logt'])) |
#                  (pd.notnull(OCs['logt_vandenbergh'])) |
#                  (pd.notnull(OCs['t[Gyr]']))
                 
                
    if (pd.notnull(row['log_age'])): #MWSC
        logAge.append(row['log_age'])
    elif (pd.notnull(row['logt'])): #Solaris
        logAge.append(row['logt'])
    elif (pd.notnull(row['log(t[yr])K'])): #Kharchenko
        logAge.append(row['log(t[yr])K'])
    elif (pd.notnull(row['Age'])): #WEBDA
        logAge.append(np.log10(row['Age']))

    #mass
#          (pd.notnull(OCs['logM[MSun]'])) |
#          (pd.notnull(OCs['num_cluster_stars'])) |
#          (pd.notnull(OCs['Stars'])) | 
#          (pd.notnull(OCs['logMA[MSun]'])) |
#          (pd.notnull(OCs['N1sr0'])) |
#          (pd.notnull(OCs['N1sr1'])) |
#          (pd.notnull(OCs['N1sr2'])) 
        
    if (pd.notnull(row['logM[Msun]'])): #Piskunov
        logMass.append(row['logM[Msun]'])    
    elif (pd.notnull(row['num_cluster_stars'])): #MWSC <-- NEED TO FIX THIS 
        logMass.append(np.log10(row['num_cluster_stars']*meanM))
        
print(len(name), len(logAge), len(logMass))

### Make a few plots

In [ ]:
f,(ax1, ax2) = plt.subplots(1,2)

ax1.hist(logAge, bins=40, density=True)
ax1.set_xlabel('log(Age [yr?])')
ax1.set_yscale('log')

ax2.hist(logMass, bins=40, density=True)
ax2.set_xlabel('log(Mass [Msun])')
ax2.set_yscale('log')

In [ ]:
df = pd.read_csv("OCcompiled_hasAgeMass.csv")

data = np.vstack((df['logAge'].values, df['logMass'].values))
KDE = gaussian_kde(data)
sample = KDE.resample(size=int(1e5))

nbins = 40

f,(ax1, ax2) = plt.subplots(1,2)
ax1.hist(df['logAge'].values, bins=nbins, density=True)
ax1.hist(sample[0,:], bins=nbins, density=True, histtype='step')
ax1.set_xlabel('log(Age [yr?])')
ax1.set_yscale('log')

ax2.hist(df['logMass'].values, bins=nbins, density=True)
ax2.hist(sample[1,:], bins=nbins, density=True, histtype='step')
ax2.set_xlabel('log(Mass [Msun])')
ax2.set_yscale('log')

lt = 5
lm = 2
values = np.vstack([lt, lm])
print(KDE(values))
#NOTE: the age KDE seems to be missing the edges.  Maybe I should set those to zero automatically?